## Preparación del entorno

In [1]:
# Importar bibliotecas necesarias para el análisis
import pandas as pd

## Carga de datos

### Delitos en la Ciudad
El dataset utilizado en este análisis proviene del [Portal de Datos Abiertos del Gobierno de la Ciudad de Buenos Aires](https://data.buenosaires.gob.ar/dataset/delitos). El dataset abarca el período 2016-2023 Y los datos están organizados en archivos separados por año, donde cada archivo contiene la información de los delitos reportados en ese período específico. Los datos están disponibles bajo la licencia Creative Commons Attribution.

El dataset incluye las siguientes variables:

|Nombre|Tipo|Descripción|
|------|----|-----------|
|id_mapa|integer|Identificador único.|
|anio|date|Año en el que se registró el evento.|
|mes|string|Mes en que ocurrió el evento.|
|dia|string|Día de la semana en que ocurrió el evento.|
|fecha|date|Fecha exacta del evento.|
|franja|integer|Franja horaria en la que ocurrió el evento.|
|tipo|string|Clasificación del tipo de delito.|
|subtipo|string|Subtipo del delito, más específico.|
|uso_arma|boolean|Indicador de uso de arma.|
|uso_moto|boolean|Indicador de uso de moto en el evento.|
|barrio|string|Barrio donde ocurrió el evento.|
|comuna|integer|Comuna donde ocurrió el evento.|
|latitud|float|Latitud geográfica donde ocurrió el evento.|
|longitud|float|Longitud geográfica donde ocurrió el evento.|
|cantidad|integer|Número de eventos registrados en esa ubicación y fecha.|

In [2]:
# Lista de archivos que hay que importar (delitos desde 2016 hasta 2023)
# Los archivos están almacenados en la misma carpeta que el notebook Jupyter
archivos_csv = [
    "delitos_2016.csv",
    "delitos_2017.csv",
    "delitos_2018.csv",
    "delitos_2019.csv",
    "delitos_2020.csv",
    "delitos_2021.csv",
    "delitos_2022.csv",
    "delitos_2023.csv"
]

# Crear lista para almacenar todos los data frames
dfs = []

# Importar los archivos de delitos uno por uno
for arch in archivos_csv:
    df = pd.read_excel(arch)
    dfs.append(df)

# Crear el data frame final de delitos
df_delitos = pd.concat(dfs, ignore_index=True) 

# Mostrar primeras filas del data frame de delitos
df_delitos.head()
    

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad,id-sum
0,500001.0,2016,enero,MAR,2016-01-26,21.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34617667936.0,-585309609989999.0,1,NaN
1,500004.0,2016,enero,MIE,2016-01-20,16.0,Robo,Robo total,NO,NO,VILLA REAL,10.0,-34620261641.0,-58530737721.0,1,NaN
2,500007.0,2016,enero,DOM,2016-01-03,13.0,Robo,Robo total,SI,NO,LINIERS,9.0,-3464009416108100.0,-585298256047288.0,1,NaN
3,500010.0,2016,enero,SAB,2016-01-09,17.0,Robo,Robo total,NO,NO,LINIERS,9.0,-3464009416108100.0,-585298256047288.0,1,NaN
4,500013.0,2016,enero,LUN,2016-01-25,18.0,Robo,Robo total,NO,NO,LINIERS,9.0,-3464009416108100.0,-585298256047288.0,1,NaN


### Población de la Ciudad
Para calcular la métrica de "cantidad de delitos por 100.000 personas", es necesario conocer la población de la Ciudad de Buenos Aires a lo largo de los años. Esta información se obtuvo del Instituto de Estadística y Censos de la Ciudad Autónoma de Buenos Aires. Se puede encontrar la información en el siguiente enlace: [Población total estimada por sexo, superficie y densidad poblacional según comuna. Ciudad de Buenos Aires. Años 2006/2023](https://www.estadisticaciudad.gob.ar/eyc/?p=76599)

In [ ]:
# Crear diccionario que va a almacenar la población de la Ciudad en cada año
poblacion_ciudad = {
    2016: 3059122,
    2017: 3063728,
    2018: 3068043,
    2019: 3072029,
    2020: 3075646,
    2021: 3078836,
    2022: 3081550,
    2023: 3083770
}